# Operating on Data in Pandas

One of the essential pieces of NumPy is the ability to perform quick element-wise operations, both with basic arithmetic (addition, subtraction, multiplication, etc.) and with more sophisticated operations (trigonometric functions, exponential and logarithmic functions, etc.).
**Pandas inherits much of this functionality from NumPy, and the ufuncs** that we introduced in [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) are key to this.

Pandas includes a couple useful twists, however: for unary operations like negation and trigonometric functions, these ufuncs will *preserve index and column labels* in the output, and for binary operations such as **addition and multiplication, Pandas will automatically *align indices* when passing the objects to the ufunc.**
This means that keeping the context of data and combining data from different sources–both potentially error-prone tasks with raw NumPy arrays–become essentially foolproof ones with Pandas.
We will additionally see that there are well-defined operations between one-dimensional ``Series`` structures and two-dimensional ``DataFrame`` structures.

## Ufuncs: Index Preservation

Because Pandas is designed to work with NumPy, any NumPy ufunc will work on Pandas ``Series`` and ``DataFrame`` objects.
Let's start by defining a simple ``Series`` and ``DataFrame`` on which to demonstrate this:

In [44]:
import pandas as pd
import numpy as np

In [45]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0,10,4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [46]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

A  B  C  D
0  6  9  2  6
1  7  4  3  7
2  7  2  5  4

If we apply a NumPy ufunc on either of these objects, the result will be another Pandas object *with the indices preserved:*

In [47]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [48]:
#help(np.exp)
#np.linspace(-2.945367829, 3.63245671 ,14)

Or, for a slightly more complex calculation:

In [49]:
np.sin(df * np.pi /4)

A             B         C             D
0 -1.000000  7.071068e-01  1.000000 -1.000000e+00
1 -0.707107  1.224647e-16  0.707107 -7.071068e-01
2 -0.707107  1.000000e+00 -0.707107  1.224647e-16

Any of the ufuncs discussed in [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) can be used in a similar manner.

## UFuncs: Index Alignment

For binary operations on two ``Series`` or ``DataFrame`` objects, Pandas will align indices in the process of performing the operation.
This is very convenient when working with incomplete data, as we'll see in some of the examples that follow.

### Index alignment in Series

As an example, suppose we are combining two different data sources, and find only the top three US states by *area* and the top three US states by *population*:

In [50]:
area = pd.Series({'Alaska': 1723337,
                  'Texas': 695662,
                  'California': 423967},
                 name='area')

population = pd.Series({'California': 38332521,
                        'Texas': 26448193,
                        'New York': 19651127},
                       name='population')

Let's see what happens when we divide these to compute the population density:

In [51]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

The resulting array contains the *union* of indices of the two input arrays, which could be determined using standard Python set arithmetic on these indices:

In [52]:
area.index | population.index

C:\Users\Admin\AppData\Local\Temp\ipykernel_7024\3572280633.py:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  area.index | population.index


Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

In [53]:
area.index.union(population.index)

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Any item for which one or the other does not have an entry is marked with ``NaN``, or "Not a Number," which is how Pandas marks missing data (see further discussion of missing data in [Handling Missing Data](03.04-Missing-Values.ipynb)).
This index matching is implemented this way for any of Python's built-in arithmetic expressions; any missing values are filled in with NaN by default:

In [54]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

In [55]:
""""
   A       B
0  2      NaN  8    2
1  4     1
2  6     3
3  NaN 8  5  5

"""

'"\n   A       B\n0  2      NaN  8    2\n1  4     1\n2  6     3\n3  NaN 8  5  5\n\n'

If using NaN values is not the desired behavior, the fill value can be modified using appropriate object methods in place of the operators.
For example, calling ``A.add(B)`` is equivalent to calling ``A + B``, but allows optional explicit specification of the fill value for any elements in ``A`` or ``B`` that might be missing:

In [56]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Index alignment in DataFrame

A similar type of alignment takes place for *both* columns and indices when performing operations on ``DataFrame``s:

In [57]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

A   B
0  1  11
1  5   1

In [58]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

B  A  C
0  4  0  9
1  5  8  0
2  9  2  6

In [59]:
A + B

A     B   C
0   1.0  15.0 NaN
1  13.0   6.0 NaN
2   NaN   NaN NaN

Notice that indices are aligned correctly irrespective of their order in the two objects, and indices in the result are sorted.
As was the case with ``Series``, we can use the associated object's arithmetic method and pass any desired ``fill_value`` to be used in place of missing entries.
Here we'll fill with the mean of all values in ``A`` (computed by first stacking the rows of ``A``):

In [60]:
A

A   B
0  1  11
1  5   1

In [61]:
pd.DataFrame(A.stack())

0
0 A   1
  B  11
1 A   5
  B   1

In [62]:
fill = A.stack().mean()
nuevo = A.add(B, fill_value=fill)

In [68]:
nuevo

A     B     C
0   1.0  15.0  13.5
1  13.0   6.0   4.5
2   6.5  13.5  10.5

In [33]:
A['C'] = 7
A

A   B  C
0  1  11  7
1  5   1  7

In [37]:
A.stack().mean()

5.333333333333333

In [40]:
A.mean()

A    3.0
B    6.0
C    7.0
dtype: float64

The following table lists Python operators and their equivalent Pandas object methods:

| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Ufuncs: Operations Between DataFrame and Series

When performing operations between a ``DataFrame`` and a ``Series``, the index and column alignment is similarly maintained.
Operations between a ``DataFrame`` and a ``Series`` are similar to operations between a two-dimensional and one-dimensional NumPy array.
Consider one common operation, where we find the difference of a two-dimensional array and one of its rows:

In [38]:
A = rng.randint(10, size=(3, 4))
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [39]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

According to NumPy's broadcasting rules (see [Computation on Arrays: Broadcasting](02.05-Computation-on-arrays-broadcasting.ipynb)), subtraction between a two-dimensional array and one of its rows is applied row-wise.

In Pandas, the convention similarly operates row-wise by default:

In [40]:
df = pd.DataFrame(A, columns=list('QRST'))
print(df)
df - df.iloc[0]

   Q  R  S  T
0  3  8  2  4
1  2  6  4  8
2  6  1  3  8


Q  R  S  T
0  0  0  0  0
1 -1 -2  2  4
2  3 -7  1  4

In [42]:
df.iloc[list(df["Q"] > 1)]
#mask = [True, False, True]
#df.iloc[mask]

Q  R  S  T
0  3  8  2  4
1  2  6  4  8
2  6  1  3  8

In [50]:
df["Q"] > 1

0    False
1     True
2    False
Name: Q, dtype: bool

If you would instead like to operate column-wise, you can use the object methods mentioned earlier, while specifying the ``axis`` keyword:

In [61]:
df.subtract(df['R'], axis=0)

Q  R  S  T
0 -4  0  0  4
1 -2  0 -4  4
2 -8  0 -6 -2

In [ ]:
# x       x       x        x
# 3       2       1        0
# 10^3    10^2    10^1     10^0
# 1000    100      10      1

# decimal 4293  = 4 * 1000 + 2 * 100 +  9 * 10 + 3 * 1
# en binario
# x        x        x        x
# 3        2        1        0
# 2^3      2^2      2^1      2^0
# 8        4        2        1

# en binario
#0111  0*8 + 1*4 + 1*2 + 1*1 = 7

In [59]:
oct(8)
hex(10)

'0xa'

In [51]:
df.subtract(df.iloc[0])

Q  R  S  T
0  0  0  0  0
1 -6  5  3 -2
2 -2  3  9  0

In [49]:
df

Q  R  S  T
0  7  2  0  3
1  1  7  3  1
2  5  5  9  3

Note that these ``DataFrame``/``Series`` operations, like the operations discussed above, will automatically align  indices between the two elements:

In [44]:
df

Q  R  S  T
0  7  2  0  3
1  1  7  3  1
2  5  5  9  3

In [62]:
halfrow = df.iloc[0, ::2]
halfrow

Q    1
S    5
Name: 0, dtype: int32

In [54]:
df - halfrow

Q   R    S   T
0  0.0 NaN  0.0 NaN
1 -6.0 NaN  3.0 NaN
2 -2.0 NaN  9.0 NaN

This preservation and alignment of indices and columns means that operations on data in Pandas will always maintain the data context, which prevents the types of silly errors that might come up when working with heterogeneous and/or misaligned data in raw NumPy arrays.